In [2]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../functions.jl")

display_transform (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [3]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Kinematic computations 

In [4]:
m = compile(robot)
kcache = new_kinematics_cache(m)  

rh_palm_frame_id = get_compiled_frameID(m, "rh_palm")
palm_transform = get_transform(kcache, rh_palm_frame_id)

Transform{Float64}([0.0, -0.01, 0.24701], Rotor{Float64}(1.0, [0.0, 0.0, 0.0]))

### Hand Visualization

In [5]:
# Target transform 
R = [1.0 0.0 0.0; 
     0.0 1.0 0.0;
     0.0 0.0 1.0]

t = [0.00, -0.01, 0.31701]  # Example translation vector

# Construct the homogeneous transformation matrix
T = [R t; 0 0 0 1]

display(T)  # Show the matrix

4×4 Matrix{Float64}:
 1.0  0.0  0.0   0.0
 0.0  1.0  0.0  -0.01
 0.0  0.0  1.0   0.31701
 0.0  0.0  0.0   1.0

In [6]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)
display_frame(m, ls, "rh_palm")
display_transform(ls, T)

# Display figure
display(fig)

GLMakie.Screen(...)

In [215]:
show(IOContext(stdout, :limit => false),  MIME("text/plain"), coordinates(robot))

Dict{String, VMRobotControl.CoordinateData} with 55 entries:
  "rh_lfdistal_mass_coord" => FramePoint{Float64, String}(rh_lfdistal, [0.0, 0.0, 0.012])
  "rh_rfdistal_mass_coord" => FramePoint{Float64, String}(rh_rfdistal, [0.0, 0.0, 0.012])
  "rh_thdistal_mass_coord" => FramePoint{Float64, String}(rh_thdistal, [0.0, 0.0, 0.01375])
  "rh_rftip_mass_coord" => FrameOrigin{String}(rh_rftip)
  "rh_palm_mass_coord" => FramePoint{Float64, String}(rh_palm, [0.0, 0.0, 0.035])
  "rh_rfknuckle_mass_coord" => FrameOrigin{String}(rh_rfknuckle)
  "rh_wrist_mass_coord" => FramePoint{Float64, String}(rh_wrist, [0.0, 0.0, 0.029])
  "rh_ffdistal_inertia_coord" => FrameAngularVelocity{String}("rh_ffdistal")
  "rh_thhub_mass_coord" => FrameOrigin{String}(rh_thhub)
  "rh_rfmiddle_mass_coord" => FramePoint{Float64, String}(rh_rfmiddle, [0.0, 0.0, 0.0125])
  "rh_mftip_mass_coord" => FrameOrigin{String}(rh_mftip)
  "rh_ffdistal_mass_coord" => FramePoint{Float64, String}(rh_ffdistal, [0.0, 0.0, 0.012])
  "rh_m

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [7]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Creation of the sphere

In [8]:
vm = Mechanism{Float64}("Sphere + VirtualTrack")
root_vm = root_frame(vm)
F1 = add_frame!(vm; id="sphere_L1_frame")
F2 = add_frame!(vm; id="sphere_L2_frame")
F_sphere = add_frame!(vm; id="sphere_frame")

J1 = Prismatic(SVector(1. ,0. ,0. ))
J2 = Prismatic(SVector(0. ,1. ,0. ))
J3 = Prismatic(SVector(0. ,0. ,1. ))

add_joint!(vm, J1; parent=root_vm, child=F1, id="J1")
add_joint!(vm, J2; parent=F1, child=F2, id="J2")
add_joint!(vm, J3; parent=F2, child=F_sphere, id="J3")

add_coordinate!(vm, FrameOrigin(F_sphere); id = "sphere position")
add_component!(vm, PointMass(1.0, "sphere position"); id="sphere mass")
add_gravity_compensation!(vm, VMRobotControl.DEFAULT_GRAVITY)

add_coordinate!(vm, JointSubspace("J1"); id="J1")
add_coordinate!(vm, JointSubspace("J2"); id="J2")
add_coordinate!(vm, JointSubspace("J3"); id="J3")

"J3"

### Creation of the rail

In [9]:
start = SVector(0.0, -0.2, 0.32)

Nk = 6
L = 0.11
y0 = start[2]
y_vector = Vector(LinRange(y0, y0+L, Nk))
spline_points = Matrix{Float64}(undef, 0, 3)
for i = 1:Nk
    spline_points = vcat(
        spline_points,
        hcat(start[1], y_vector[i], start[3])
    )
end

spline = CubicSpline(spline_points)

cart_frame = add_frame!(vm, "Cart")
add_joint!(vm, Rail(spline, zero(Transform{Float64}));
        parent=root_frame(vm), child=cart_frame,        id="RailJoint")

add_coordinate!(vm, JointSubspace("RailJoint");         id="CartDistance")
add_coordinate!(vm, FrameOrigin(cart_frame);            id="Target position")
add_component!(vm, LinearInerter(1.0, "Target position");  id="CartInertance") # Cart mass
add_component!(vm, LinearDamper(100.0, "Target position"); id="CartDamper");

### Addition of the multiples springs/dampers

Hand motion

In [10]:
vms = VirtualMechanismSystem("myShadowVMS", robot, vm)
root = root_frame(vms.robot)

# FINGER SPACING --> PRE-SHAPING
add_coordinate!(vms, CoordDifference(".robot.rh_ffmiddle_mass_coord", ".robot.rh_mfmiddle_mass_coord"); id="ff mf middle diff")
add_coordinate!(vms, CoordDifference(".robot.rh_mfmiddle_mass_coord", ".robot.rh_rfmiddle_mass_coord"); id="mf rf middle diff")
add_coordinate!(vms, CoordDifference(".robot.rh_rfmiddle_mass_coord", ".robot.rh_lfmiddle_mass_coord"); id="rf lf middle diff")
add_coordinate!(vms, CoordDifference(".robot.rh_ffmiddle_mass_coord", ".robot.rh_thmiddle_mass_coord"); id="ff th middle diff")

add_coordinate!(vms, CoordSlice("ff mf middle diff", SVector(1,3)); id = "ff mf middle diff xz")
add_coordinate!(vms, CoordSlice("mf rf middle diff", SVector(1,3)); id = "mf rf middle diff xz")
add_coordinate!(vms, CoordSlice("rf lf middle diff", SVector(1,3)); id = "rf lf middle diff xz")

add_coordinate!(vms, CoordNorm("ff mf middle diff xz"); id="ff mf middle planar norm")
add_coordinate!(vms, CoordNorm("mf rf middle diff xz"); id="mf rf middle planar norm")
add_coordinate!(vms, CoordNorm("rf lf middle diff xz"); id="rf lf middle planar norm")
add_coordinate!(vms, CoordNorm("ff th middle diff"); id="ff th middle norm")

fingers_spacing = 0.03
add_coordinate!(vms, ConstCoord(fingers_spacing); id = "fingers spacing")
thumb_spacing = 0.1
add_coordinate!(vms, ConstCoord(thumb_spacing); id = "thumb spacing")

add_coordinate!(vms, CoordDifference("ff mf middle planar norm", "fingers spacing"); id = "ff mf middle planar error")
add_coordinate!(vms, CoordDifference("mf rf middle planar norm", "fingers spacing"); id = "mf rf middle planar error")
add_coordinate!(vms, CoordDifference("rf lf middle planar norm", "fingers spacing"); id = "rf lf middle planar error")
add_coordinate!(vms, CoordDifference("ff th middle norm", "thumb spacing"); id = "ff th middle error")


K = 100.0
D = 10.0 

add_component!(vms, LinearSpring(K, "ff mf middle planar error");     id="ff mf middle spring")
add_component!(vms, LinearDamper(D, "ff mf middle planar error");     id="ff mf middle damper")
add_component!(vms, LinearSpring(K, "mf rf middle planar error");     id="mf rf middle spring")
add_component!(vms, LinearDamper(D, "mf rf middle planar error");     id="mf rf middle damper")
add_component!(vms, LinearSpring(K, "rf lf middle planar error");     id="rf lf middle spring")
add_component!(vms, LinearDamper(D, "rf lf middle planar error");     id="rf lf middle damper")
add_component!(vms, LinearSpring(K, "ff th middle error");     id="ff th middle spring")
add_component!(vms, LinearDamper(D, "ff th middle error");     id="ff th middle damper")

#add_component!(vms, LinearSpring(100.0, ".robot.rh_LFJ5_coord"); id = "lf j5 angular spring")

# TARGET REACHING -->
#add_coordinate!(vms,  ConstCoord(SVector(0.0, -0.075, 0.32));  id="Target position")

add_coordinate!(vms, CoordDifference(".robot.rh_fftip_mass_coord", ".virtual_mechanism.Target position"); id="ff position error")
add_coordinate!(vms, CoordDifference(".robot.rh_mftip_mass_coord", ".virtual_mechanism.Target position"); id="mf position error")
add_coordinate!(vms, CoordDifference(".robot.rh_rftip_mass_coord", ".virtual_mechanism.Target position"); id="rf position error")
add_coordinate!(vms, CoordDifference(".robot.rh_lftip_mass_coord", ".virtual_mechanism.Target position"); id="lf position error")
add_coordinate!(vms, CoordDifference(".robot.rh_thtip_mass_coord", ".virtual_mechanism.Target position"); id="th position error")

K_matrix = SMatrix{3, 3}(0., 0., 0., 0., 0., 0., 0., 0., 0.)
D_matrix = SMatrix{3, 3}(0., 0., 0., 0., 0.0, 0., 0., 0., 0.)

add_component!(vms, LinearSpring(K_matrix, "ff position error"); id="ff spring")
add_component!(vms, LinearDamper(D_matrix, "ff position error"); id="ff damper")
add_component!(vms, LinearSpring(K_matrix, "mf position error"); id="mf spring")
add_component!(vms, LinearDamper(D_matrix, "mf position error"); id="mf damper")
add_component!(vms, LinearSpring(K_matrix, "rf position error"); id="rf spring")
add_component!(vms, LinearDamper(D_matrix, "rf position error"); id="rf damper")
add_component!(vms, LinearSpring(K_matrix, "lf position error"); id="lf spring")
add_component!(vms, LinearDamper(D_matrix, "lf position error"); id="lf damper")
add_component!(vms, LinearSpring(K_matrix, "th position error"); id="th spring")
add_component!(vms, LinearDamper(D_matrix, "th position error"); id="th damper")

max_power = 10.0
force_source = ForceSource(SVector(0.0), max_power, ".virtual_mechanism.CartDistance")
add_component!(vms, force_source;   id="Cart force source");

Repulsive ball

In [11]:
sphere_radius = 0.035

add_coordinate!(vms, ConstCoord(sphere_radius); id="sphere radius")

add_coordinate!(vms, FramePoint(".robot.rh_palm", SVector(0. , 0., 0.07)); id="second palm point")

repulsed_frames = (".robot.rh_fftip_mass_coord", ".robot.rh_mftip_mass_coord", ".robot.rh_rftip_mass_coord",".robot.rh_lftip_mass_coord" , 
                    ".robot.rh_thtip_mass_coord", ".robot.rh_ffmiddle_mass_coord",".robot.rh_mfmiddle_mass_coord", ".robot.rh_rfmiddle_mass_coord",
                    ".robot.rh_lfmiddle_mass_coord",  ".robot.rh_thmiddle_mass_coord", ".robot.rh_ffproximal_mass_coord", ".robot.rh_mfproximal_mass_coord",
                    ".robot.rh_rfproximal_mass_coord", ".robot.rh_lfproximal_mass_coord", ".robot.rh_thproximal_mass_coord", ".robot.rh_palm_mass_coord", "second palm point")
frames_names = ("fftip", "mftip", "rftip", "lftip", "thtip", "ffmiddle", "mfmiddle", "rfmiddle", "lfmiddle", "thmiddle", "ffprox", 
                "mfprox", "rfprox", "lfprox", "thprox", "palm", "palm2")

for i in 1:length(repulsed_frames)
    frame = repulsed_frames[i]
    add_coordinate!(vms, CoordDifference(frame, ".virtual_mechanism.sphere position") ; id = "$(frames_names[i]) sphere diff" )
    add_coordinate!(vms, CoordNorm("$(frames_names[i]) sphere diff") ; id = "$(frames_names[i]) sphere diff norm")
    add_coordinate!(vms, CoordDifference("$(frames_names[i]) sphere diff norm", "sphere radius"); id = "shifted $(frames_names[i]) sphere diff norm" )

    add_component!(vms, ReLUSpring(1000.0, "shifted $(frames_names[i]) sphere diff norm", true); id="$(frames_names[i]) sphere repulsive spring")
    add_component!(vms, RectifiedDamper(100.0, "$(frames_names[i]) sphere diff norm", (0.0, 1.1*sphere_radius), true, false); id="$(frames_names[i]) sphere damper")
end

## Simulating the Robot

### Setting Up the Simulation

In [12]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

preshaping_phase = true
grasping_phase = false
flags = (preshaping_phase, grasping_phase)

function f_setup(cache) 
    preshaping_springs = ("ff mf middle spring", "mf rf middle spring", "rf lf middle spring", "ff th middle spring")
    preshaping_dampers = ("ff mf middle damper", "mf rf middle damper", "rf lf middle damper", "ff th middle damper")
    preshaping_springs_ID = []
    preshaping_dampers_ID = []

    for (spring, damper) in zip(preshaping_springs, preshaping_dampers)
        spring_ID = get_compiled_componentID(dcache, spring)
        damper_ID = get_compiled_componentID(dcache, damper)
    
        push!(preshaping_springs_ID, spring_ID)
        push!(preshaping_dampers_ID, damper_ID)
    end

    grasping_springs = ("ff spring", "mf spring", "rf spring", "lf spring", "th spring")
    grasping_dampers = ("ff damper", "mf damper", "rf damper", "lf damper", "th damper")
    grasping_springs_ID = []
    grasping_dampers_ID = []

    for (spring, damper) in zip(grasping_springs, grasping_dampers)
        spring_ID = get_compiled_componentID(dcache, spring)
        damper_ID = get_compiled_componentID(dcache, damper)
    
        push!(grasping_springs_ID, spring_ID)
        push!(grasping_dampers_ID, damper_ID)
    end
    
    return (preshaping_springs_ID , preshaping_dampers_ID, grasping_springs_ID, grasping_dampers_ID, get_compiled_componentID(cache, "Cart force source"), flags)
end

function f_control(cache, t, args, extra)
    preshaping_springs_ID, preshaping_dampers_ID , grasping_springs_ID, grasping_dampers_ID, force_source_id, flags = args
    preshaping_phase, grasping_phase = flags
    
    # Move from pre-shaping to grasping phase
    if t > 3 && preshaping_phase

        #flags update
        preshaping_phase = false
        grasping_phase = true

        # for (spring_id, damper_id) in zip(preshaping_springs_ID, preshaping_dampers_ID)
        #     cache[spring_id] = remake(cache[spring_id]; stiffness = 0.0)
        #     cache[damper_id] = remake(cache[damper_id]; damping = 0.0)
        # end

        #Only removing the long spring between the thumb and the first finger
        cache[preshaping_springs_ID[4]] = remake(cache[preshaping_springs_ID[4]]; stiffness = 0.0)
        cache[preshaping_dampers_ID[4]] = remake(cache[preshaping_dampers_ID[4]]; damping = 0.0)

        #activating the target reaching springs
        new_K_matrix = SMatrix{3, 3}(100., 0., 0., 0., 100., 0., 0., 0., 100.)
        new_D_matrix = SMatrix{3, 3}(30., 0., 0., 0., 30., 0., 0., 0., 30.)

        for (spring_id, damper_id) in zip(grasping_springs_ID, grasping_dampers_ID)
            cache[spring_id] = remake(cache[spring_id]; stiffness = new_K_matrix)
            cache[damper_id] = remake(cache[damper_id]; damping = new_D_matrix)
        end

        #activating the force source
        cache[force_source_id] = remake(cache[force_source_id] ; force_max = SVector(1.0))
    end 

    nothing
end

tspan = (0., 8.)
vms_compiled = compile(vms)

sphere_initial_position = [0.0, -0.055, 0.32]
q_vm_init = vcat(sphere_initial_position, 0.0)
q = (zero_q(vms_compiled.robot), q_vm_init) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot with repulsive ball"

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-6, reltol=1e-6);

[ Info: Simulating shadow robot with repulsive ball
ODE   0%|█                                              |  ETA: N/A
ODE   3%|██                                             |  ETA: 0:01:04
ODE   7%|████                                           |  ETA: 0:00:45
ODE  10%|█████                                          |  ETA: 0:00:38
ODE  14%|███████                                        |  ETA: 0:00:33
ODE  18%|█████████                                      |  ETA: 0:00:31
ODE  21%|██████████                                     |  ETA: 0:00:29
ODE  25%|████████████                                   |  ETA: 0:00:27
ODE  28%|██████████████                                 |  ETA: 0:00:25
ODE  32%|████████████████                               |  ETA: 0:00:24
ODE  36%|█████████████████                              |  ETA: 0:00:22
ODE  39%|███████████████████                            |  ETA: 0:00:21
ODE  42%|████████████████████                           |  ETA: 0:00:21
ODE  44%|███████

### Visualizing the Results

In [13]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [-0.25, -0.6, 0.62]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))
plotting_vm_kcache = map(plotting_kcache) do k
    VMRobotControl.virtual_mechanism_cache(k)
end
cartID = get_compiled_coordID(plotting_kcache[], ".virtual_mechanism.Target position")

robotvisualize!(ls, plotting_kcache)
scatter!(ls, plotting_kcache, cartID; color=:red, marker=:rect, markersize=5)
robotsketch!(ls, plotting_vm_kcache; scale = 0.1)

# Adding the sphere to the visual
sphere_pos_point3f_observable = map(plotting_kcache) do cache
    sphere_pos = Point3f(configuration(cache, get_compiled_coordID(plotting_kcache[], ".virtual_mechanism.sphere position")))
end
sphere = Sphere(Point3f(SVector(0.,0.,0.)), sphere_radius)
sphere_mesh = mesh!(ls, sphere ; color=:blue, alpha=1.0, transparency=true)
on(sphere_pos_point3f_observable) do sphere_pos_point
    translate!(sphere_mesh, sphere_pos_point...)
end

savepath = joinpath(module_path, "docs/src/assets/shadowhand_two_phases_ball_grasp.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);

### Save Trajectory into CSV 

In [17]:
using CSV, Tables

function save_solution_to_csv(sol, filename, joint_names, vms_compiled; rate=nothing, time_vector=nothing)
    # Generate time vector based on rate or use the provided one
    if time_vector === nothing
        if rate === nothing
            error("Either 'rate' or 'time_vector' must be provided.")
        end
        t_min, t_max = sol.t[begin], sol.t[end]
        time_vector = collect(t_min:1/rate:t_max)  # Uniform sampling
    end

    # Prepare header (positions and velocities)
    pos_names = [joint_names[i] * "_pos" for i in 1:length(joint_names)]
    vel_names = [joint_names[i] * "_vel" for i in 1:length(joint_names)]
    header = ["time"; pos_names; vel_names]

    # Initialize data array with time
    data = time_vector
    pos_data = []
    vel_data = []

    n = length(joint_names)

    # Loop through each joint and collect its values at all time steps
    for joint_name in joint_names
        # Get the joint ID from the compiled robot
        jointID = get_compiled_jointID(vms_compiled, ".robot." * joint_name)
        joint = vms_compiled[jointID]

        # Get the indices for position and velocity
        joint_index = q_idxs(joint)[1]

        joint_positions = [sol(t)[joint_index] for t in time_vector]  # Positions
        joint_velocities = [sol(t)[n + joint_index] for t in time_vector]  # Velocities

        # First iteration 
        if length(pos_data) == 0
            pos_data = joint_positions
            vel_data = joint_velocities
        else
            pos_data = hcat(pos_data, joint_positions)
            vel_data = hcat(vel_data,joint_velocities)
        end
    end

    data = hcat(data, pos_data, vel_data)

    CSV.write(filename, Tables.table(data); header=header)

    println("CSV saved to $filename")
end


joint_names = ["rh_WRJ1", "rh_WRJ2", "rh_FFJ1", "rh_FFJ2", "rh_FFJ3", "rh_FFJ4", "rh_MFJ1",
                "rh_MFJ2", "rh_MFJ3", "rh_MFJ4", "rh_RFJ1", "rh_RFJ2", "rh_RFJ3", "rh_RFJ4", "rh_LFJ1",
                "rh_LFJ2", "rh_LFJ3", "rh_LFJ4", "rh_LFJ5", "rh_THJ1", "rh_THJ2", "rh_THJ3",
                "rh_THJ4", "rh_THJ5"]


save_solution_to_csv(sol, "shadowhand_solution.csv" , joint_names, vms_compiled; rate=10)


CSV saved to shadowhand_solution.csv
